# iCAT Overlay
---

#### Packages

In [ ]:
from pathlib import Path
from itertools import product
from pprint import pprint

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import altair as alt

from shapely.geometry import box
from shapely import affinity
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import renderapi
from renderapi.tilespec import TileSpec
from renderapi.transform import AffineModel as AffineRender

from icatapi.render_pandas import *
from icatapi.overlay import *

#### Settings

In [ ]:
# pandas display settings
# -----------------------
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 20)

## Set up `render-ws` environment
---

In [ ]:
# `render` project parameters
# ---------------------------
owner = 'rlane'  # replace with your name
project = '20191230_RL010'

# Create a renderapi.connect.Render object
# ----------------------------------------
render_connect_params = {
    'host': 'sonic',
    'port': 8080,
    'owner': owner,
    'project': project,
    'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
    'memGB': '2G'
}
render = renderapi.connect(**render_connect_params)

# Infer stack and section info
# ----------------------------
stacks = renderapi.render.get_stacks_by_owner_project(render=render)
stacks_EM = [stack for stack in stacks if 'EM' in stack]
stacks_FM = [stack for stack in stacks if 'EM' not in stack]
stacks_2_overlay = stacks_FM
stack_underneath = 'big_EM'

# Output
# ------
out = f"""\
all stacks.......... {stacks}
EM stacks........... {stacks_EM}
FM stacks........... {stacks_FM}
stacks to overlay... {stacks_2_overlay}
stack underneath.... {stack_underneath}
...
"""
print(out)

# Create project DataFrame
# ------------------------
df_project = create_project_DataFrame(render=render)
df_project.groupby('stack')\
          .apply(lambda x: x.sample(3))\
          .drop(['camera', 'temca'], axis=1)

## Compute overlay transforms
---

### Sample overlay transform

In [ ]:
# Choose random FM image tile filepath
fp = Path(df_project.loc[df_project['stack'].isin(stacks_FM + [stack_underneath])]\
                    .sample(1)['imageUrl']\
                    .str.split('://')\
                    .iloc[0][1])

# Test parser
(psx, psy), ro, sh, (trx, try_) = get_transform_metadata(fp)

# Output
out = f"""\
File.......... {fp.parent.name}/{fp.name}
Pixel size.... ({1e9*psx:.1f}nm, {1e9*psy:.1f}nm)
Rotation...... {np.rad2deg(ro):.1f}°
Shear......... {sh:.2e}
Translation... ({1e6*trx:.1f}um, {1e6*try_:.1f}um)
"""
print(out)

### Compile transform data

In [ ]:
# Create multiindex
overlay_cols = ['stack', 'z', 'sectionId',
                'tileId', 'imageRow', 'imageCol',
                'width', 'height',
                'imageUrl', 'transforms',
                'ps', 'ro', 'sh', 'tr',
                'overlay_transforms']
multiindex = np.array(list(product(overlay_cols,
                                   ['EM', 'FM']))).T.tolist()
# Initialize overlay DataFrame
df_overlay = pd.DataFrame(columns=multiindex)

# Loop through overlay stacks
for stack in stacks_2_overlay:
    # Filter to stack DataFrame
    df_stack = df_project.loc[(df_project['stack'] == stack)].copy()

    # Loop through FM tiles
    for i, FM_tile in df_stack.iterrows():

        # Find correlative EM tile
        EM_tile = df_project.loc[(df_project['stack'] == stack_underneath) &\
                                 (df_project['z'] == FM_tile['z']) &\
                                 (df_project['imageRow'] == FM_tile['imageRow']) &\
                                 (df_project['imageCol'] == FM_tile['imageCol'])].iloc[0]

        # Parse tif for transform metadata
        fp_EM = Path(EM_tile['imageUrl'].split('://')[1])
        fp_FM = Path(FM_tile['imageUrl'].split('://')[1])
        ps_EM, ro_EM, sh_EM, tr_EM = get_transform_metadata(fp_EM)
        ps_FM, ro_FM, sh_FM, tr_FM = get_transform_metadata(fp_FM)

        # Build up overlay DataFrame one FM tile at a time
        df_overlay.loc[i, 'stack'] = EM_tile['stack'], FM_tile['stack']
        df_overlay.loc[i, 'z'] = EM_tile['z'], FM_tile['z']
        df_overlay.loc[i, 'sectionId'] = EM_tile['sectionId'], FM_tile['sectionId']
        df_overlay.loc[i, 'tileId'] = EM_tile['tileId'], FM_tile['tileId']
        df_overlay.loc[i, 'imageRow'] = EM_tile['imageRow'], FM_tile['imageRow']
        df_overlay.loc[i, 'imageCol'] = EM_tile['imageCol'], FM_tile['imageCol']
        df_overlay.loc[i, 'width'] = EM_tile['width'], FM_tile['width']
        df_overlay.loc[i, 'height'] = EM_tile['height'], FM_tile['height']
        df_overlay.loc[i, 'imageUrl'] = EM_tile['imageUrl'], FM_tile['imageUrl']
        df_overlay.loc[i, 'transforms'] = [EM_tile['transforms']], [FM_tile['transforms']]
        # Transform metadata
        df_overlay.loc[i, 'ps'] = ps_EM, ps_FM
        df_overlay.loc[i, 'ro'] = ro_EM, ro_FM
        df_overlay.loc[i, 'sh'] = sh_EM, sh_FM
        df_overlay.loc[i, 'tr'] = tr_EM, tr_FM

# Unpack nested transforms
df_overlay[('transforms', 'EM')] = df_overlay[('transforms', 'EM')].apply(lambda x: x[0])
df_overlay[('transforms', 'FM')] = df_overlay[('transforms', 'FM')].apply(lambda x: x[0])

# Preview tile data
pd.set_option('display.max_colwidth', 15)
df_overlay.groupby(('stack', 'FM'))\
          .apply(lambda x: x.sample(4))\
          .drop(['ps', 'ro', 'sh', 'tr',
                 'overlay_transforms'], level=0, axis=1)

In [ ]:
# Preview transform data
pd.set_option('display.max_colwidth', 15)
df_overlay.groupby(('stack', 'FM'))\
          .apply(lambda x: x.sample(4))\
          .drop(['tileId', 'width', 'height', 'imageUrl',
                 'transforms', 'overlay_transforms'], level=0, axis=1)

### Compute overlay transforms

In [ ]:
# Iterate through CLEM tiles
for i, tile in df_overlay.iterrows():

    # Initial translation derived from stage position
    T0_EM = tile[('transforms', 'EM')][-1]
    T0_FM = tile[('transforms', 'FM')][-1]

    # Overlay translation to center tiles
    C0_EM = AffineRender(B0=-tile[('width', 'EM')]/2,
                         B1=-tile[('height', 'EM')]/2)
    C0_FM = AffineRender(B0=-tile[('width', 'FM')]/2,
                         B1=-tile[('height', 'FM')]/2)

    # Compute relative transform
    A = AffineRender()
    A.M = compute_relative_transform(*np.array(tile['ps'].tolist()).T,
                                     *np.array(tile['ro'].tolist()),
                                     *np.array(tile['sh'].tolist()),
                                     *np.array(tile['tr'].tolist()).T)

    # Set overlay transforms
    df_overlay.loc[i, 'overlay_transforms'] = [C0_EM, T0_EM], [C0_FM, A, T0_EM]

# Preview sample overlay transform
sample = df_overlay.sample(1)
out = f"""\
{sample[('tileId', 'EM')].iloc[0]} | \
{sample[('tileId', 'FM')].iloc[0]}
"""
print(out)
sample['overlay_transforms'].values.tolist()

## Upload overlaid stacks to `render-ws`
---

### Create overlaid stacks

In [ ]:
# Collect overlay stacks
stacks_overlaid = []

# Iterate through stacks
for stack in tqdm(stacks_2_overlay + [stack_underneath]):

    # Set overlay stack name
    stack_overlaid = f"{stack}_overlaid"
    stacks_overlaid.append(stack_overlaid)

    # Get stack DataFrame
    df_stack = df_project.loc[(df_project['stack'] == stack)].copy()
    df_stack.rename(columns={'minIntensity': 'minint',
                             'maxIntensity': 'maxint'}, inplace=True)

    # Create `TileSpec`s
    tile_specs = []
    for i, tile in df_stack.iterrows():

        # Get overlay transforms for each tile from overlay DataFrame
        k = 'FM' if stack in stacks_FM else 'EM' 
        tile['tforms'] = df_overlay.loc[(df_overlay[('stack', k)] == tile['stack']) &\
                                        (df_overlay[('z', k)] == tile['z']) &\
                                        (df_overlay[('imageRow', k)] == tile['imageRow']) &\
                                        (df_overlay[('imageCol', k)] == tile['imageCol']),
                                        ('overlay_transforms', k)].iloc[0]

        # Create `TileSpec
        ts = TileSpec(**tile.to_dict())
        tile_specs.append(ts)

    # Create stack
    renderapi.stack.create_stack(stack_overlaid,
                                 render=render)

    # Import TileSpecs to render
    renderapi.client.import_tilespecs(stack_overlaid,
                                      tile_specs,
                                      render=render)

    # Set stack state to complete
    renderapi.stack.set_stack_state(stack_overlaid,
                                    'COMPLETE',
                                    render=render)

## Inspect overlaid stacks
---

### Tile map

In [ ]:
# Specify stacks and sections
stacks_2_plot = stacks_overlaid
sections_2_plot = df_project['sectionId'].unique().tolist()

# Set up figure
ncols = len(sections_2_plot)
fig, axes = plt.subplots(ncols=ncols, figsize=(8*ncols, 8))
axmap = {k: v for k, v in zip(sections_2_plot, axes.flat)}
cmap = {k: v for k, v in zip(stacks_2_plot, sns.color_palette(n_colors=len(stacks_2_plot)))}

# Iterate through layers
df_stacks = create_stacks_DataFrame(stacks_2_plot,
                                    render=render)
for sectionId, layer in tqdm(df_stacks.groupby('sectionId')):
    # Collect all tiles in each layer to determine bounds
    boxes = []
    # Set axis
    ax = axmap[sectionId]

    # Loop through tilesets within each layer
    for stack, tileset in layer.groupby('stack'):

        # Loop through each tile
        for i, tile in tileset.iterrows():

            # Create `shapely.box` resembling raw image tile
            b = box(0, 0, tile['width'], tile['height'])
            # Apply transforms to `shapely.box`
            for tform in tile['transforms']:
                A = (tform.M[:2, :2].ravel().tolist() +
                     tform.M[:2,  2].ravel().tolist())
                b = affinity.affine_transform(b, A)
            boxes.append(b)
            # Get coordinates of `shapely.box` to plot matplotlib polygon patch
            xy = np.array(b.exterior.xy).T
            p = Polygon(xy, color=cmap[stack], alpha=0.2)
            ax.add_patch(p)

    # Axis aesthetics
    ax.set_title(sectionId)
    ax.set_xlabel('X [px]')
    ax.set_ylabel('Y [px]')
    # Determine bounds
    bounds = np.swapaxes([b.exterior.xy for b in boxes], 1, 2).reshape(-1, 2)
    ax.set_xlim(bounds[:, 0].min(), bounds[:, 0].max())
    ax.set_ylim(bounds[:, 1].min(), bounds[:, 1].max())
    ax.invert_yaxis()
    ax.set_aspect('equal')

### Render images

In [ ]:
# Specify stacks and sections
stacks_2_plot = stacks_overlaid
sections_2_plot = df_project['sectionId'].unique().tolist()

# Set up figure
nrows = len(stacks_2_plot)
ncols = len(sections_2_plot)
fig, axes = plt.subplots(nrows, ncols, figsize=(8*ncols, 8*nrows))
axmap = {k: v for k, v in zip(product(stacks_2_plot, sections_2_plot), axes.flat)}
cmap = {k: v for k, v in zip(stacks_2_plot, sns.color_palette(n_colors=len(stacks_2_plot)))}

# Iterate through layers
df_stacks = create_stacks_DataFrame(stacks_2_plot,
                                    render=render)
for (stack, sectionId), tileset in tqdm(df_stacks.groupby(['stack', 'sectionId'])):

    # Set axis
    ax = axmap[(stack, sectionId)]
    # Fetch tileset image
    z = tileset['z'].iloc[0]
    bounds = renderapi.stack.get_bounds_from_z(stack=stack,
                                               z=z,
                                               render=render)
    scale = 1024 / np.max([bounds['maxX'] - bounds['minX'],
                           bounds['maxY'] - bounds['minY']])
    image = renderapi.image.get_section_image(stack=stack,
                                              z=z,
                                              scale=scale,
                                              maxTileSpecsToRender=30,
                                              render=render)
    # Plot
    extent = [bounds['minX'],  # left
              bounds['maxX'],  # right
              bounds['minY'],  # bottom
              bounds['maxY']]  # top
    ax.imshow(image, extent=extent, origin='lower')
    # Axis aesthetics
    ax.set_title(f"{stack} | {sectionId}")
    ax.set_xlabel('X [px]')
    ax.set_ylabel('Y [px]')
    ax.set_xlim()
    ax.invert_yaxis()